In [9]:
import sys
sys.path.append('C:\Workspace\PathBench\src')

In [10]:
from algorithms.configuration.configuration import Configuration
from simulator.services.services import Services
import copy
from simulator.simulator import Simulator
from structures import Point
from algorithms.basic_testing import BasicTesting
from algorithms.algorithm import Algorithm
import tracemalloc
from typing import TYPE_CHECKING, List, Tuple, Type, Any, Dict, Union, Optional
from algorithms.configuration.maps.map import Map


In [11]:
from algorithms.classic.sample_based.rrt import RRT
from algorithms.classic.sample_based.rrt_star import RRT_Star
from algorithms.classic.sample_based.rrt_connect import RRT_Connect

In [12]:
algorithm_dict = {"RRT": (RRT, BasicTesting, ([], {})),
"RRT*": (RRT_Star, BasicTesting, ([], {})),
"RRT-Connect": (RRT_Connect, BasicTesting, ([], {})),}

In [13]:
algorithm_names = ["RRT", "RRT*", "RRT-Connect"]
algorithms = [(*algorithm_dict[n], n) for n in algorithm_names]

In [14]:
algorithms[1]

(algorithms.classic.sample_based.rrt_star.RRT_Star,
 algorithms.basic_testing.BasicTesting,
 ([], {}),
 'RRT*')

In [15]:
for idx, (algorithm_type, testing_type, algo_params, algoname) in enumerate(algorithms):
    print("Algorithm Type: {}\nTesting Type: {}\nAlgorithm Params: {}\nAlgorithm Name: {}".format(
            algorithm_type, testing_type, algo_params, algoname), end="\n\n")

algorithm_results: List[Dict[str, Any]] = [None for _ in range(len(algorithms))]

Algorithm Type: <class 'algorithms.classic.sample_based.rrt.RRT'>
Testing Type: <class 'algorithms.basic_testing.BasicTesting'>
Algorithm Params: ([], {})
Algorithm Name: RRT

Algorithm Type: <class 'algorithms.classic.sample_based.rrt_star.RRT_Star'>
Testing Type: <class 'algorithms.basic_testing.BasicTesting'>
Algorithm Params: ([], {})
Algorithm Name: RRT*

Algorithm Type: <class 'algorithms.classic.sample_based.rrt_connect.RRT_Connect'>
Testing Type: <class 'algorithms.basic_testing.BasicTesting'>
Algorithm Params: ([], {})
Algorithm Name: RRT-Connect



In [16]:
maps: List[Map] = []
for i in range(3):
    maps.append("/testing_maps_pickles/block_map_1000/" + str(i))
    # maps.append("C://Workspace/PathBench/data/maps/testing_maps_pickles/block_map_1000/" + str(i))

In [17]:
def __run_simulation(grid: Map, algorithm_type: Type[Algorithm], testing_type: Type[BasicTesting],
                        algo_params: Tuple[list, dict], agent_pos: Point = None) -> Dict[str, Any]:
    config = Configuration()
    config.simulator_initial_map = copy.deepcopy(grid)
    config.simulator_algorithm_type = algorithm_type
    config.simulator_testing_type = testing_type
    config.simulator_algorithm_parameters = algo_params

    if agent_pos:
        config.simulator_initial_map.move_agent(agent_pos, True, False)

    sim: Simulator = Simulator(Services(config))

    tracemalloc.start()

    resu = sim.start().get_results()

    _, peak = tracemalloc.get_traced_memory()  # current, peak

    tracemalloc.stop()

    resu['memory'] = (peak/1000)

    return resu

In [18]:
algorithms

[(algorithms.classic.sample_based.rrt.RRT,
  algorithms.basic_testing.BasicTesting,
  ([], {}),
  'RRT'),
 (algorithms.classic.sample_based.rrt_star.RRT_Star,
  algorithms.basic_testing.BasicTesting,
  ([], {}),
  'RRT*'),
 (algorithms.classic.sample_based.rrt_connect.RRT_Connect,
  algorithms.basic_testing.BasicTesting,
  ([], {}),
  'RRT-Connect')]

In [19]:
for idx, (algorithm_type, testing_type, algo_params, algoname) in enumerate(algorithms):
    results: List[Dict[str, Any]] = []
    for _, grid in enumerate(maps):
        results.append(__run_simulation(grid, algorithm_type, testing_type, algo_params))

TypeError: type 'function' is not an acceptable base type

In [30]:
results[15]

IndexError: list index out of range

In [5]:
code_string = "class PathPlanning(SampleBasedAlgorithm):\n    def __init__(self, services: Services, testing: BasicTesting = None) -> None:\n        super().__init__(services, testing)\n        # Additional member to track if initial solution found\n        self._solution_found: bool = False\n        # To store best cost found so far (initialized large)\n        self._best_cost: float = float('inf')\n        # Cache start and goal vertex for convenience\n        self._start_vertex: Vertex = self._graph.root_vertex_start\n        self._goal_vertex: Vertex = Vertex(self._get_grid().goal.position)\n        self._goal_vertex.cost = float('inf')\n        # Prepare display info container\n        self._init_displays()\n    \n    def set_display_info(self) -> List[MapDisplay]:\n        return super().set_display_info()\n    \n    def _heuristic_distance(self, p1: Point, p2: Point) -> float:\n        return torch.norm(p1.to_tensor() - p2.to_tensor()).item()\n    \n    def _informed_sample(self, c_best: float, c_min: float, start: Point, goal: Point) -> Point:\n        \"\"\"\n        Sample within prolate hyperspheroid defined by start, goal, and current best cost.\n        If no solution yet (c_best == inf), sample uniformly.\n        \"\"\"\n        if c_best == float('inf'):\n            return self._get_random_sample()\n\n        # Compute center and coordinate transforms for ellipsoid sampling\n        start_np = np.array(start)\n        goal_np = np.array(goal)\n        c_min = float(c_min)\n        c_best = float(c_best)\n\n        center = (start_np + goal_np) / 2\n\n        # Unit vector along start to goal\n        direction = (goal_np - start_np)\n        norm_dir = np.linalg.norm(direction)\n        if norm_dir == 0:\n            return Point(*goal_np)\n\n        e1 = direction / norm_dir\n        # Create orthonormal basis using SVD or Gram-Schmidt for 2D\n        # In 2D, the orthonormal is just perpendicular vector\n        if e1.size == 2:\n            e2 = np.array([-e1[1], e1[0]])\n            basis = np.vstack((e1, e2)).T  # Columns are basis vectors\n        else:\n            # fallback to identity (should be 2D anyway)\n            basis = np.eye(len(e1))\n\n        # Radii of the ellipsoid along each axis\n        r1 = c_best / 2\n        r2 = math.sqrt(c_best**2 - c_min**2) / 2\n\n        # Sample random point inside unit ball in 2D\n        while True:\n            sample_unit = np.random.randn(2)\n            sample_unit /= np.linalg.norm(sample_unit)\n            if np.random.rand() <= 1.0:\n                break\n        # Uniform scale for 2D unit circle\n        scale = np.random.rand() ** 0.5\n        sample_ball = sample_unit * scale\n\n        # Stretch and rotate sample to ellipsoid\n        sample_ellipsoid = np.array([r1 * sample_ball[0], r2 * sample_ball[1]])\n        sample_world = center + basis @ sample_ellipsoid\n\n        # Clip to grid and make sure it is valid\n        sample_point = Point(*np.clip(sample_world, 0, self._get_grid().size.n_dim - 1).astype(int))\n        if self._get_grid().is_agent_valid_pos(sample_point):\n            return sample_point\n        else:\n            # fallback uniform sample if invalid\n            return self._get_random_sample()\n\n    def _extract_path(self, q_new: Vertex) -> None:\n        \"\"\"\n        Similar to RRT* extract path but moves agent with key frames\n        \"\"\"\n        path: List[Vertex] = []\n        current = q_new\n        while current is not None:\n            path.append(current)\n            if len(current.parents) == 0:\n                break\n            # Pick first parent (RRT* is tree)\n            current = next(iter(current.parents), None)\n        path.reverse()\n\n        for v in path:\n            self.move_agent(v.position)\n            grid = self._get_grid()\n            if isinstance(grid, RosMap):\n                grid.publish_wp(grid.agent.position)\n            self.key_frame(ignore_key_frame_skip=True)\n    \n    def _find_path_internal(self) -> None:\n\n        max_dist_init: float = 10.0\n        max_dist_min: float = 3.0\n        max_dist_decay: float = 0.9995  # decay max_dist slowly over iterations to more refined local exploration\n\n        max_radius: float = 50.0\n        lambda_rrt_star: float = 50.0\n        dimension = 2\n        iterations: int = 10000\n\n        start_time = time.time()\n\n        c_min = self._heuristic_distance(self._start_vertex.position, self._goal_vertex.position)\n        c_best = float('inf')\n\n        for i in range(iterations):\n            elapsed = time.time() - start_time\n            if elapsed > 30.0:\n                # Timeout reached, stop searching and treat as no path found\n                break\n\n            # Use informed sampling once a solution is found, else uniform random sampling\n            if self._solution_found:\n                q_sample = self._informed_sample(c_best, c_min, self._start_vertex.position, self._goal_vertex.position)\n            else:\n                q_sample = self._get_random_sample()\n\n            q_nearest = self._get_nearest_vertex(q_sample)\n            if q_nearest.position == q_sample:\n                continue\n\n            # Adaptive max extension distance shrinking with iterations\n            max_dist = max(max_dist_min, max_dist_init * (max_dist_decay ** i))\n            q_new = self._get_new_vertex(q_nearest, q_sample, max_dist)\n\n            if not self._get_grid().is_valid_line_sequence(self._get_grid().get_line_sequence(q_nearest.position, q_new.position)):\n                continue\n\n            card_v = torch.tensor(float(self._graph.size))\n            log_card_v = torch.log(card_v)\n            radius = min(lambda_rrt_star * ((log_card_v / card_v) ** (1 / dimension)), max_radius)\n            Q_near = self._get_vertices_within_radius(q_new, radius)\n            q_min = q_nearest\n            c_min_cost = q_nearest.cost + self._heuristic_distance(q_nearest.position, q_new.position)\n\n            for q_near in Q_near:\n                if self._get_grid().is_valid_line_sequence(self._get_grid().get_line_sequence(q_near.position, q_new.position)):\n                    cost_candidate = q_near.cost + self._heuristic_distance(q_near.position, q_new.position)\n                    if cost_candidate < c_min_cost:\n                        q_min = q_near\n                        c_min_cost = cost_candidate\n\n            dist_parent_child = self._heuristic_distance(q_min.position, q_new.position)\n            q_new.cost = q_min.cost + dist_parent_child\n            self._graph.add_edge(q_min, q_new)\n\n            # Rewiring\n            for q_near in Q_near:\n                if self._get_grid().is_valid_line_sequence(self._get_grid().get_line_sequence(q_new.position, q_near.position)):\n                    cost_through_new = q_new.cost + self._heuristic_distance(q_new.position, q_near.position)\n                    if cost_through_new < q_near.cost:\n                        q_parent = None\n                        for parent in q_near.parents:\n                            q_parent = parent\n                            break\n                        self._graph.remove_edge(q_parent, q_near)\n                        q_near.cost = None\n                        q_near.cost = cost_through_new\n                        self._graph.add_edge(q_new, q_near)\n\n            # Check if new vertex is in goal region\n            if self._get_grid().is_agent_in_goal_radius(agent_pos=q_new.position):\n                # Update solution found status\n                cost_to_goal = q_new.cost + self._heuristic_distance(q_new.position, self._goal_vertex.position)\n                if cost_to_goal < c_best:\n                    c_best = cost_to_goal\n                    self._solution_found = True\n                    self._extract_path(q_new)\n                    # Break after first found path improves the solution\n                    break\n\n            self.key_frame() "

In [18]:
import ast
from pprint import pprint

In [15]:
astcode = ast.parse(code_string, mode='exec')

In [21]:
pprint(astcode)

In [37]:
code = '''
for i in range(10):
    print(add(i, i+1))
'''

In [38]:
astcode = ast.parse(code, mode='exec')

In [ ]:
ast.NodeTransformer

In [42]:
print(ast.dump(astcode))

Module(body=[For(target=Name(id='i', ctx=Store()), iter=Call(func=Name(id='range', ctx=Load()), args=[Constant(value=10, kind=None)], keywords=[]), body=[Expr(value=Call(func=Name(id='print', ctx=Load()), args=[Call(func=Name(id='add', ctx=Load()), args=[Name(id='i', ctx=Load()), BinOp(left=Name(id='i', ctx=Load()), op=Add(), right=Constant(value=1, kind=None))], keywords=[])], keywords=[]))], orelse=[], type_comment=None)], type_ignores=[])


In [44]:
astcode.body

In [39]:
pprint(ast.dump(astcode))

("Module(body=[For(target=Name(id='i', ctx=Store()), "
 "iter=Call(func=Name(id='range', ctx=Load()), args=[Constant(value=10, "
 "kind=None)], keywords=[]), body=[Expr(value=Call(func=Name(id='print', "
 "ctx=Load()), args=[Call(func=Name(id='add', ctx=Load()), args=[Name(id='i', "
 "ctx=Load()), BinOp(left=Name(id='i', ctx=Load()), op=Add(), "
 'right=Constant(value=1, kind=None))], keywords=[])], keywords=[]))], '
 'orelse=[], type_comment=None)], type_ignores=[])')


In [19]:
pprint(code_string)

('class PathPlanning(SampleBasedAlgorithm):\n'
 '    def __init__(self, services: Services, testing: BasicTesting = None) -> '
 'None:\n'
 '        super().__init__(services, testing)\n'
 '        # Additional member to track if initial solution found\n'
 '        self._solution_found: bool = False\n'
 '        # To store best cost found so far (initialized large)\n'
 "        self._best_cost: float = float('inf')\n"
 '        # Cache start and goal vertex for convenience\n'
 '        self._start_vertex: Vertex = self._graph.root_vertex_start\n'
 '        self._goal_vertex: Vertex = Vertex(self._get_grid().goal.position)\n'
 "        self._goal_vertex.cost = float('inf')\n"
 '        # Prepare display info container\n'
 '        self._init_displays()\n'
 '    \n'
 '    def set_display_info(self) -> List[MapDisplay]:\n'
 '        return super().set_display_info()\n'
 '    \n'
 '    def _heuristic_distance(self, p1: Point, p2: Point) -> float:\n'
 '        return torch.norm(p1.to_tensor(

In [10]:
pp = PATHPLANNING()

[2025-06-24 19:46:04] - '.pathbench.json' not found, falling back to default state data
[2025-06-24 19:46:04] - Active torch device: cpu
Replacing object DenseMap
[2025-06-24 19:46:04] - Loaded [C:\Workspace\PathBench/data/maps/testing_maps_pickles/block_map_1000/0]
[2025-06-24 19:46:04] - Saving state to '.pathbench.json'
[2025-06-24 19:46:04] - Saving state to '.pathbench.json'
[2025-06-24 19:46:04] - Saving state to '.pathbench.json'
[2025-06-24 19:46:04] - Algorithm <class 'algorithms.classic.graph_based.a_star.AStar'> started..
[2025-06-24 19:46:04] - Original distance: 8.06
[2025-06-24 19:46:04] - Occupancy percentage: 14.84%
[2025-06-24 19:46:04] - Goal was FOUND
[2025-06-24 19:46:04] - Total steps: 7
[2025-06-24 19:46:04] - Total distance: 8.66
[2025-06-24 19:46:04] - Trajectory Smoothness: 0.20
[2025-06-24 19:46:04] - Total time: 0.001467 seconds
[2025-06-24 19:46:04] - Search space percentage (no fringe): 5.08%
[2025-06-24 19:46:04] - Fringe percentage: 12.89%
[2025-06-24 19:

In [11]:
pp.evaluate(code_string)

[2025-06-24 19:46:33] - Active torch device: cpu
Replacing object DenseMap
[2025-06-24 19:46:33] - Loaded [C:\Workspace\PathBench/data/maps/testing_maps_pickles/block_map_1000/0]
Error: 'PathPlanning' object has no attribute '_graph'


In [29]:
code_string = "def adddd(a, b):\n    return a + b\n\nprint(adddd(1, 2))"

In [31]:
import types
import sys
heuristic_module = types.ModuleType("heuristic_module")
                
# Execute the code string in the new module's namespace
a = exec(code_string, heuristic_module.__dict__)

# Add the module to sys.modules so it can be imported
sys.modules[heuristic_module.__name__] = heuristic_module

fitness, std_reward = evaluate(heuristic_module, 150)


3


NameError: name 'evaluate' is not defined

In [33]:
print(a)

None


In [27]:
heuristic_module = types.ModuleType("heuristic_module")


NameError: name 'types' is not defined

In [10]:
heuristic_module.__dict__

{'__name__': 'heuristic_module',
 '__doc__': None,
 '__package__': None,
 '__loader__': None,
 '__spec__': None}

In [13]:
sys.modules

{'sys': <module 'sys' (built-in)>,
 'builtins': <module 'builtins' (built-in)>,
 '_frozen_importlib': <module 'importlib._bootstrap' (frozen)>,
 '_imp': <module '_imp' (built-in)>,
 '_warnings': <module '_warnings' (built-in)>,
 '_frozen_importlib_external': <module 'importlib._bootstrap_external' (frozen)>,
 '_io': <module 'io' (built-in)>,
 'marshal': <module 'marshal' (built-in)>,
 'nt': <module 'nt' (built-in)>,
 '_thread': <module '_thread' (built-in)>,
 '_weakref': <module '_weakref' (built-in)>,
 'winreg': <module 'winreg' (built-in)>,
 'time': <module 'time' (built-in)>,
 'zipimport': <module 'zipimport' (frozen)>,
 '_codecs': <module '_codecs' (built-in)>,
 'codecs': <module 'codecs' from 'c:\\Users\\byeonghwa\\anaconda3\\envs\\path_planning\\lib\\codecs.py'>,
 'encodings.aliases': <module 'encodings.aliases' from 'c:\\Users\\byeonghwa\\anaconda3\\envs\\path_planning\\lib\\encodings\\aliases.py'>,
 'encodings': <module 'encodings' from 'c:\\Users\\byeonghwa\\anaconda3\\envs\\p

In [1]:
import sys
sys.path.append('C:\Workspace\EoH_Path_planning')

In [9]:
from eoh.methods.eoh.eoh_evolution import Evolution
from eoh.problems.optimization.path_planning.prompts import GetPrompts
from eoh.problems.optimization.path_planning.run import PATHPLANNING
from eoh.methods.eoh.classic_planning_method import GetPlanningCode

In [3]:
e = Evolution('1', '2', '3', '4', '5', '6', GetPrompts())

In [9]:
get_code = GetPlanningCode()

In [10]:
code = get_code.rrt_connect

In [15]:
pop1 = {'algorithm':'RRT-Connect', 'code':code}

In [16]:
e.get_prompt_time(pop1)

"You are given a reference implementations for path planning algorithms on a discrete grid environment.\nYour task is to design and implement an **improved path planning algorithm**, written as a Python class named `PathPlanning`, that is inspired by but not limited to the provided examples.\nI have one algorithm with its code as follows. Algorithm description: RRT-Connect\nCode:\n\nfrom typing import List\n\nimport torch\nimport numpy as np\n\nfrom memory_profiler import profile\n\nfrom algorithms.classic.sample_based.core.sample_based_algorithm import SampleBasedAlgorithm\nfrom algorithms.basic_testing import BasicTesting\nfrom simulator.services.services import Services\nfrom structures import Point\n\nfrom algorithms.classic.sample_based.core.vertex import Vertex\nfrom algorithms.classic.sample_based.core.graph import gen_forest, Forest\n\nn=0\n\nclass RRT_Connect(SampleBasedAlgorithm):\n    _graph: Forest\n    _max_dist: float\n    _iterations: int\n\n    def __init__(self, servic

In [17]:
print("You are given a reference implementations for path planning algorithms on a discrete grid environment.\nYour task is to design and implement an **improved path planning algorithm**, written as a Python class named `PathPlanning`, that is inspired by but not limited to the provided examples.\nI have one algorithm with its code as follows. Algorithm description: RRT-Connect\nCode:\n\nfrom typing import List\n\nimport torch\nimport numpy as np\n\nfrom memory_profiler import profile\n\nfrom algorithms.classic.sample_based.core.sample_based_algorithm import SampleBasedAlgorithm\nfrom algorithms.basic_testing import BasicTesting\nfrom simulator.services.services import Services\nfrom structures import Point\n\nfrom algorithms.classic.sample_based.core.vertex import Vertex\nfrom algorithms.classic.sample_based.core.graph import gen_forest, Forest\n\nn=0\n\nclass RRT_Connect(SampleBasedAlgorithm):\n    _graph: Forest\n    _max_dist: float\n    _iterations: int\n\n    def __init__(self, services: Services, testing: BasicTesting = None) -> None:\n        super().__init__(services, testing)\n        \n        self._graph = gen_forest(self._services, Vertex(self._get_grid().agent.position), Vertex(self._get_grid().goal.position), [])\n        self._graph.edges_removable = False\n        self._init_displays()\n        \n        self._max_dist = 10\n        self._iterations = 10000\n\n    # Helper Functions #\n    # -----------------#\n\n    def _extend(self, root_vertex: Vertex, q: Point) -> str:\n        self._q_near: Vertex = self._get_nearest_vertex(root_vertex, q)\n        self._q_new: Vertex = self._get_new_vertex(self._q_near, q, self._max_dist)\n        if self._get_grid().is_valid_line_sequence(self._get_grid().get_line_sequence(self._q_near.position, self._q_new.position)):\n            self._graph.add_edge(self._q_near, self._q_new)\n            if self._q_new.position == q:\n                return 'reached'\n            else:\n                return 'advanced'\n        return 'trapped'\n\n    def _connect(self, root_vertex: Vertex, q: Vertex) -> str:\n        S = 'advanced'\n        while S == 'advanced':\n            S = self._extend(root_vertex, q.position)\n        self._mid_vertex = q\n        return S\n\n    def _extract_path(self):\n\n        # trace back\n        path_mid_to_b: List[Vertex] = [self._q_new]\n\n        while len(path_mid_to_b[-1].parents) != 0:\n            for parent in path_mid_to_b[-1].parents:\n                path_mid_to_b.append(parent)\n                break\n\n        path_a_to_mid: List[Vertex] = [self._extension_target]\n\n        while len(path_a_to_mid[-1].parents) != 0:\n            for parent in path_a_to_mid[-1].parents:\n                path_a_to_mid.append(parent)\n                break\n\n        path_a_to_mid.reverse()\n        path = path_a_to_mid + path_mid_to_b\n\n        if self._graph.root_vertices[0] is self._graph.root_vertex_goal:\n            path.reverse()\n\n        for p in path:\n            self.move_agent(p.position)\n            self.key_frame(ignore_key_frame_skip=True)\n\n    def _get_random_sample(self) -> Point:\n        while True:\n            rand_pos = np.random.randint(0, self._get_grid().size, self._get_grid().size.n_dim)\n            sample: Point = Point(*rand_pos)\n            if self._get_grid().is_agent_valid_pos(sample):\n                return sample\n\n    def _get_nearest_vertex(self, graph_root_vertex: Vertex, q_sample: Point) -> Vertex:\n        return self._graph.get_nearest_vertex([graph_root_vertex], q_sample)\n\n    def _get_new_vertex(self, q_near: Vertex, q_sample: Point, max_dist) -> Vertex:\n        dir = q_sample.to_tensor() - q_near.position.to_tensor()\n        if torch.norm(dir) <= max_dist:\n            return Vertex(q_sample)\n\n        dir_normalized = dir / torch.norm(dir)\n        q_new = Point.from_tensor(q_near.position.to_tensor() + max_dist * dir_normalized)\n        return Vertex(q_new)\n\n    # Overridden Implementation #\n    # --------------------------#\n\n    #@profile\n    def _find_path_internal(self) -> None:\n\n        for i in range(self._iterations):\n            global n \n            n+=1\n\n            if n == 100:\n                break\n\n            q_rand: Point = self._get_random_sample()\n\n            if not self._extend(self._graph.root_vertices[0], q_rand) == 'trapped':\n                self._extension_target = self._q_new\n                if self._connect(self._graph.root_vertices[-1], self._q_new) == 'reached':\n                    self._extract_path()\n                    break\n            self._graph.reverse_root_vertices()\n\n            # visualization code\n            self.key_frame()\n\nPlease help us create a new algorithm with improved time by modifying the provided algorithm. \nIdentify the backbone idea in the provided algorithms.\n### Constraints:\n- Please write a brief description of the algorithm you generated.\n- The description must be inside a brace.\n- Implement it in Python.\n- Your class must be named `PathPlanning`.\n- It must inherit from `SampleBasedAlgorithm`.\n- It should work with existing components: `Forest`, `Point`, `Vertex`, etc.\n- Include core methods like `_extend`, `_connect`, `_extract_path`, `_find_path_internal`.\n### Reusable helper functions (optional to modify or extend):\n- `_get_random_sample()`\n- `_get_nearest_vertex(...)`\n- `_get_new_vertex(...)`\n- `_get_grid()`\n\n### You may freely define new helper functions if necessary\n- If your approach benefits from additional utility methods (e.g., cost estimation, region sampling, custom distance functions), feel free to create and use them.\n### The `_find_path_internal` function is the main function executed for path planning.\n\n\n### Constraints:\n- Please write a brief description of the algorithm you generated.\n- The description must be inside a brace.\n- Implement it in Python.\n- Your class must be named `PathPlanning`.\n- It must inherit from `SampleBasedAlgorithm`.\n- It should work with existing components: `Forest`, `Point`, `Vertex`, etc.\n- Include core methods like `_extend`, `_connect`, `_extract_path`, `_find_path_internal`.\n### Reusable helper functions (optional to modify or extend):\n- `_get_random_sample()`\n- `_get_nearest_vertex(...)`\n- `_get_new_vertex(...)`\n- `_get_grid()`\n\n### You may freely define new helper functions if necessary\n- If your approach benefits from additional utility methods (e.g., cost estimation, region sampling, custom distance functions), feel free to create and use them.\n### The `_find_path_internal` function is the main function executed for path planning.\n\nYou may assume access to the new implementation via the baseline class and its methods, such as `_get_random_sample`, `_get_nearest_vertex`, etc.\nDo not give additional explanations.")

You are given a reference implementations for path planning algorithms on a discrete grid environment.
Your task is to design and implement an **improved path planning algorithm**, written as a Python class named `PathPlanning`, that is inspired by but not limited to the provided examples.
I have one algorithm with its code as follows. Algorithm description: RRT-Connect
Code:

from typing import List

import torch
import numpy as np

from memory_profiler import profile

from algorithms.classic.sample_based.core.sample_based_algorithm import SampleBasedAlgorithm
from algorithms.basic_testing import BasicTesting
from simulator.services.services import Services
from structures import Point

from algorithms.classic.sample_based.core.vertex import Vertex
from algorithms.classic.sample_based.core.graph import gen_forest, Forest

n=0

class RRT_Connect(SampleBasedAlgorithm):
    _graph: Forest
    _max_dist: float
    _iterations: int

    def __init__(self, services: Services, testing: Basic

In [19]:
output = '''
{This improved path planning algorithm is a *Bidirectional RRT-Connect with Adaptive Step Size* that enhances performance by dynamically adjusting the step size based on local sampling density and search progress. It improves connection probability by increasing step size when extensions frequently succeed, and decreasing it when they frequently fail. This reduces trapped cases and accelerates convergence, while preserving the core RRT-Connect architecture.}

import torch
import numpy as np
from typing import List

from algorithms.classic.sample_based.core.sample_based_algorithm import SampleBasedAlgorithm
from algorithms.basic_testing import BasicTesting
from simulator.services.services import Services
from structures import Point
from algorithms.classic.sample_based.core.vertex import Vertex
from algorithms.classic.sample_based.core.graph import gen_forest, Forest

class PathPlanning(SampleBasedAlgorithm):
    _graph: Forest
    _max_dist: float
    _iterations: int

    def __init__(self, services: Services, testing: BasicTesting = None) -> None:
        super().__init__(services, testing)
        self._graph = gen_forest(self._services, Vertex(self._get_grid().agent.position), Vertex(self._get_grid().goal.position), [])
        self._graph.edges_removable = False
        self._init_displays()

        self._max_dist = 10
        self._iterations = 10000
        self._success_streak = 0
        self._fail_streak = 0
        self._min_dist = 2
        self._max_allowed_dist = 20

    def _extend(self, root_vertex: Vertex, q: Point) -> str:
        q_near = self._get_nearest_vertex(root_vertex, q)
        q_new = self._get_new_vertex(q_near, q, self._max_dist)

        if self._get_grid().is_valid_line_sequence(self._get_grid().get_line_sequence(q_near.position, q_new.position)):
            self._graph.add_edge(q_near, q_new)
            self._last_q_new = q_new
            if q_new.position == q:
                self._adjust_step_size(success=True)
                return 'reached'
            else:
                self._adjust_step_size(success=True)
                return 'advanced'

        self._adjust_step_size(success=False)
        return 'trapped'

    def _connect(self, root_vertex: Vertex, q: Vertex) -> str:
        S = 'advanced'
        while S == 'advanced':
            S = self._extend(root_vertex, q.position)
        self._mid_vertex = q
        return S

    def _extract_path(self):
        path_mid_to_b: List[Vertex] = [self._last_q_new]
        while path_mid_to_b[-1].parents:
            path_mid_to_b.append(path_mid_to_b[-1].parents[0])

        path_a_to_mid: List[Vertex] = [self._extension_target]
        while path_a_to_mid[-1].parents:
            path_a_to_mid.append(path_a_to_mid[-1].parents[0])

        path_a_to_mid.reverse()
        path = path_a_to_mid + path_mid_to_b

        if self._graph.root_vertices[0] is self._graph.root_vertex_goal:
            path.reverse()

        for p in path:
            self.move_agent(p.position)
            self.key_frame(ignore_key_frame_skip=True)

    def _get_random_sample(self) -> Point:
        while True:
            rand_pos = np.random.randint(0, self._get_grid().size, self._get_grid().size.n_dim)
            sample = Point(*rand_pos)
            if self._get_grid().is_agent_valid_pos(sample):
                return sample

    def _get_nearest_vertex(self, graph_root_vertex: Vertex, q_sample: Point) -> Vertex:
        return self._graph.get_nearest_vertex([graph_root_vertex], q_sample)

    def _get_new_vertex(self, q_near: Vertex, q_sample: Point, max_dist) -> Vertex:
        dir = q_sample.to_tensor() - q_near.position.to_tensor()
        if torch.norm(dir) <= max_dist:
            return Vertex(q_sample)

        dir_normalized = dir / torch.norm(dir)
        q_new = Point.from_tensor(q_near.position.to_tensor() + max_dist * dir_normalized)
        return Vertex(q_new)

    def _adjust_step_size(self, success: bool):
        if success:
            self._success_streak += 1
            self._fail_streak = 0
        else:
            self._fail_streak += 1
            self._success_streak = 0

        if self._success_streak >= 5 and self._max_dist < self._max_allowed_dist:
            self._max_dist += 1
            self._success_streak = 0
        elif self._fail_streak >= 3 and self._max_dist > self._min_dist:
            self._max_dist -= 1
            self._fail_streak = 0

    def _find_path_internal(self) -> None:
        for _ in range(self._iterations):
            q_rand = self._get_random_sample()

            if self._extend(self._graph.root_vertices[0], q_rand) != 'trapped':
                self._extension_target = self._last_q_new
                if self._connect(self._graph.root_vertices[-1], self._last_q_new) == 'reached':
                    self._extract_path()
                    break

            self._graph.reverse_root_vertices()
            self.key_frame()
'''

In [20]:
import re
def get_alg(prompt_content):

        response = prompt_content

        algorithm = re.findall(r"\{(.*)\}", response, re.DOTALL)
        if len(algorithm) == 0:
            if 'python' in response:
                algorithm = re.findall(r'^.*?(?=python)', response,re.DOTALL)
            elif 'import' in response:
                algorithm = re.findall(r'^.*?(?=import)', response,re.DOTALL)
            else:
                algorithm = re.findall(r'^.*?(?=def)', response,re.DOTALL)

        code = re.findall(r"import.*return", response, re.DOTALL)
        if len(code) == 0:
            code = re.findall(r"def.*return", response, re.DOTALL)

        algorithm = algorithm[0]
        code = code[0] 

        code_all = code


        return [code_all, algorithm]

In [22]:
code, algorithm = get_alg(output)

In [25]:
print(code)

import torch
import numpy as np
from typing import List

from algorithms.classic.sample_based.core.sample_based_algorithm import SampleBasedAlgorithm
from algorithms.basic_testing import BasicTesting
from simulator.services.services import Services
from structures import Point
from algorithms.classic.sample_based.core.vertex import Vertex
from algorithms.classic.sample_based.core.graph import gen_forest, Forest

class PathPlanning(SampleBasedAlgorithm):
    _graph: Forest
    _max_dist: float
    _iterations: int

    def __init__(self, services: Services, testing: BasicTesting = None) -> None:
        super().__init__(services, testing)
        self._graph = gen_forest(self._services, Vertex(self._get_grid().agent.position), Vertex(self._get_grid().goal.position), [])
        self._graph.edges_removable = False
        self._init_displays()

        self._max_dist = 10
        self._iterations = 10000
        self._success_streak = 0
        self._fail_streak = 0
        self._min_dis

In [24]:
algorithm

'This improved path planning algorithm is a *Bidirectional RRT-Connect with Adaptive Step Size* that enhances performance by dynamically adjusting the step size based on local sampling density and search progress. It improves connection probability by increasing step size when extensions frequently succeed, and decreasing it when they frequently fail. This reduces trapped cases and accelerates convergence, while preserving the core RRT-Connect architecture.'

In [13]:
def get_alg_new(prompt_content):
    # 알고리즘 설명은 중괄호 내부 내용
    algorithm_match = re.search(r"\{(.*?)\}", prompt_content, re.DOTALL)
    algorithm = algorithm_match.group(1).strip() if algorithm_match else ""

    # 코드 추출: 알고리즘 설명 이후에 나오는 코드 전체
    # 방법: 중괄호로 감싼 설명 이후 첫 등장하는 `import` 또는 `class` 부터 끝까지 추출
    code_match = re.search(r"\}\s*(.*)", prompt_content, re.DOTALL)
    code = code_match.group(1).strip() if code_match else ""

    return [code, algorithm]

In [27]:
code, algorithm = get_alg_new(output)

In [28]:
print(code)

import torch
import numpy as np
from typing import List

from algorithms.classic.sample_based.core.sample_based_algorithm import SampleBasedAlgorithm
from algorithms.basic_testing import BasicTesting
from simulator.services.services import Services
from structures import Point
from algorithms.classic.sample_based.core.vertex import Vertex
from algorithms.classic.sample_based.core.graph import gen_forest, Forest

class PathPlanning(SampleBasedAlgorithm):
    _graph: Forest
    _max_dist: float
    _iterations: int

    def __init__(self, services: Services, testing: BasicTesting = None) -> None:
        super().__init__(services, testing)
        self._graph = gen_forest(self._services, Vertex(self._get_grid().agent.position), Vertex(self._get_grid().goal.position), [])
        self._graph.edges_removable = False
        self._init_displays()

        self._max_dist = 10
        self._iterations = 10000
        self._success_streak = 0
        self._fail_streak = 0
        self._min_dis

In [11]:
code = '''
 ```python

from typing import List
import torch

class PathPlanning(SampleBasedAlgorithm):
    import time
    """
    {
    This PathPlanning algorithm integrates elements from informed sampling, heuristic-guided tree growth,
    and adaptive step sizes to improve efficiency, path quality, and robustness for grid-based path planning.
    It uses a Forest structure (a tree rooted at start vertex) with the following key features:
    - Goal-biased sampling: with a probability, samples the goal itself to push the exploration towards it.
    - Heuristic cost incorporates actual path cost + Euclidean distance to goal guiding vertex selection and rewiring.
    - Adaptive max_dist: reduces the step size as the tree grows closer to the goal, enabling finer exploration near goal.
    - Rewiring step inspired by RRT*: attempts to reduce costs by reconnecting vertices within a dynamically computed radius.
    - Early stopping as soon as the goal is connected within a tolerance radius.
    - Shortcut smoothing applied after extraction to produce a shorter, smoother path.
    - Enforces a max computation time limit of 30 seconds to avoid excessive runtimes.
    This approach balances exploration and exploitation, adapts to environment complexity,
    and aims for faster convergence and better-quality paths.
    }
    """

    _graph: Forest

    def __init__(self, services, testing=None):
        super().__init__(services, testing)
        start_vertex = Vertex(self._get_grid().agent.position)
        start_vertex.cost = 0.0
        goal_vertex = Vertex(self._get_grid().goal.position)
        self._graph = gen_forest(self._services, start_vertex, goal_vertex, [])
        self._goal_radius = max(2.5, min(self._get_grid().size) * 0.05)  # radius to consider goal reached
        self._max_iterations = 3000
        self._max_time_seconds = 30.0
        self._default_max_dist = max(5.0, min(self._get_grid().size) * 0.1)
        self._lambda_rrt_star = 50.0
        self._dimension = self._get_grid().size.n_dim
        self._init_displays()

    def _heuristic_cost(self, pos: Point) -> float:
        # Heuristic: Euclidean distance to goal
        return Map.get_distance(pos, self._graph.root_vertex_goal.position)

    def _total_cost(self, cost_so_far: float, pos: Point) -> float:
        # Cost so far plus heuristic
        return cost_so_far + self._heuristic_cost(pos)

    def _adaptive_max_dist(self, dist_to_goal: float) -> float:
        # Step size decreases as closer to goal (min 1, max default)
        return max(1.0, min(self._default_max_dist, dist_to_goal / 5))

    def _get_random_sample(self) -> Point:
        # Goal bias 20% of time, otherwise uniform random valid sample
        import random
        if random.random() < 0.2:
            return self._graph.root_vertex_goal.position
        while True:
            rand_coords = [torch.randint(0, self._get_grid().size[i], (1,)).item() for i in range(self._dimension)]
            sample = Point(*rand_coords)
            if self._get_grid().is_agent_valid_pos(sample):
                return sample

    def _get_nearest_vertex(self, q_sample: Point) -> Vertex:
        return self._graph.get_nearest_vertex([self._graph.root_vertex_start], q_sample)

    def _get_vertices_within_radius(self, q_new: Vertex, radius: float) -> list:
        return self._graph.get_vertices_within_radius([self._graph.root_vertex_start], q_new.position, radius)

    def _get_new_vertex(self, q_near: Vertex, q_sample: Point, max_dist: float) -> Vertex:
        direction = q_sample.to_tensor() - q_near.position.to_tensor()
        dist = torch.norm(direction)
        if dist <= max_dist:
            return Vertex(q_sample)
        direction_normalized = direction / dist
        new_pos_tensor = q_near.position.to_tensor() + max_dist * direction_normalized
        q_new = Vertex(Point.from_tensor(new_pos_tensor))
        return q_new

    def _extract_path(self, q_new: Vertex) -> None:
        # Extract path from start to goal backward through parents, then smooth path with shortcuts
        path = []
        current = q_new

        # Add goal vertex (root_vertex_goal) connected to q_new if not already
        goal_v = self._graph.root_vertex_goal
        if goal_v not in current.children and self._get_grid().is_valid_line_sequence(self._get_grid().get_line_sequence(current.position, goal_v.position)):
            goal_v.cost = current.cost + Map.get_distance(current.position, goal_v.position)
            self._graph.add_edge(current, goal_v)

        # Walk back parents till root start
        while True:
            path.append(current.position)
            if current == self._graph.root_vertex_start or len(current.parents) == 0:
                break
            # choose the parent with lowest cost
            min_parent = None
            min_cost = float('inf')
            for parent in current.parents:
                if parent.cost is not None and parent.cost < min_cost:
                    min_cost = parent.cost
                    min_parent = parent
            if min_parent is None:
                break
            current = min_parent

        path.reverse()
        path = self._shortcut_path(path)
        for p in path:
            self.move_agent(p)
            self.key_frame(ignore_key_frame_skip=True)

    def _shortcut_path(self, path: list) -> list:
        # Basic shortcut smoothing by attempting line-of-sight skips
        if len(path) <= 2:
            return path

        smoothed_path = [path[0]]
        idx = 0
        while idx < len(path) - 1:
            next_idx = len(path) - 1
            # Find furthest point reachable directly from smoothed_path[-1]
            for test_idx in range(len(path) - 1, idx, -1):
                line_seq = self._get_grid().get_line_sequence(smoothed_path[-1], path[test_idx])
                if self._get_grid().is_valid_line_sequence(line_seq):
                    next_idx = test_idx
                    break
            smoothed_path.append(path[next_idx])
            idx = next_idx
        return smoothed_path

    def _find_path_internal(self) -> None:
        import time
        start_time = time.time()

        for iter_idx in range(self._max_iterations):
            elapsed = time.time() - start_time
            if elapsed > self._max_time_seconds:
                # Timeout: no path found within time limit
                return

            q_sample = self._get_random_sample()
            q_nearest = self._get_nearest_vertex(q_sample)
            if q_nearest.position == q_sample:
                continue

            dist_to_goal = Map.get_distance(q_nearest.position, self._graph.root_vertex_goal.position)
            max_dist = self._adaptive_max_dist(dist_to_goal)
            q_new = self._get_new_vertex(q_nearest, q_sample, max_dist)

            line_seq = self._get_grid().get_line_sequence(q_nearest.position, q_new.position)
            if not self._get_grid().is_valid_line_sequence(line_seq):
                continue

            # Cost from start to q_new via q_nearest
            cost_through_nearest = q_nearest.cost + Map.get_distance(q_nearest.position, q_new.position)
            q_new.cost = cost_through_nearest

            # Rewiring radius (Ball radius)
            card_v = float(self._graph.size + 1)
            log_card_v = torch.log(torch.tensor(card_v))
            radius = min(self._lambda_rrt_star * ((log_card_v / card_v) ** (1 / self._dimension)), self._max_dist)

            Q_near = self._get_vertices_within_radius(q_new, radius)

            # Choose parent yielding minimum cost to q_new
            q_min = q_nearest
            c_min = cost_through_nearest
            for q_near in Q_near:
                if q_near.position == q_nearest.position:
                    continue
                line_to_new = self._get_grid().get_line_sequence(q_near.position, q_new.position)
                if not self._get_grid().is_valid_line_sequence(line_to_new):
                    continue

                cost_via_q_near = q_near.cost + Map.get_distance(q_near.position, q_new.position)
                if cost_via_q_near < c_min:
                    q_min = q_near
                    c_min = cost_via_q_near

            q_new.cost = c_min
            self._graph.add_edge(q_min, q_new)

            # Rewire other vertices in neighborhood through q_new if it improves cost
            for q_near in Q_near:
                if q_near == q_min:
                    continue
                line_new_to_near = self._get_grid().get_line_sequence(q_new.position, q_near.position)
                if not self._get_grid().is_valid_line_sequence(line_new_to_near):
                    continue

                cost_via_new = q_new.cost + Map.get_distance(q_new.position, q_near.position)
                if cost_via_new < q_near.cost:
                    # Remove old parent edge
                    old_parents = list(q_near.parents)
                    for parent in old_parents:
                        self._graph.remove_edge(parent, q_near)
                    # Add new edge from q_new
                    q_near.cost = cost_via_new
                    self._graph.add_edge(q_new, q_near)

            # Check if reached goal radius
            dist_to_goal_new = Map.get_distance(q_new.position, self._graph.root_vertex_goal.position)
            if dist_to_goal_new <= self._goal_radius:
                # Connect directly to goal if possible and extract path
                if self._get_grid().is_valid_line_sequence(self._get_grid().get_line_sequence(q_new.position, self._graph.root_vertex_goal.position)):
                    self._extract_path(q_new)
                    return

            self.key_frame()
```
'''

In [2]:
import re

In [3]:
def get_alg_new(prompt_content):
    # 알고리즘 설명은 중괄호 내부 내용
    algorithm_match = re.search(r"\{(.*?)\}", prompt_content, re.DOTALL)
    algorithm = algorithm_match.group(1).strip() if algorithm_match else ""
    
    # code_block_match = re.search(r"```(?:python)?(.*?)```", prompt_content, re.DOTALL)
    # code = code_block_match.group(1).strip()
    class_name = "PathPlanning"

    class_pattern = rf"(class\s+{class_name}\s*\(.*?\):\n(?:[ \t]+.*\n)+)"
    class_match = re.search(class_pattern, prompt_content)
    code = class_match.group(1).strip() if class_match else ""

    code = re.sub(r'([\'"]{3})\s*\{.*?\}\s*\1', '', code, flags=re.DOTALL)


    return [code, algorithm]

In [12]:
import ast

def extract_class_code(code: str, class_name: str) -> str:
    module = ast.parse(code)
    for node in module.body:
        if isinstance(node, ast.ClassDef) and node.name == class_name:
            start_line = node.lineno - 1  # ast uses 1-based indexing
            end_line = node.end_lineno    # requires Python 3.8+
            lines = code.splitlines()
            return '\n'.join(lines[start_line:end_line])
    return ""

In [13]:
aaa = code.replace('```python\n', '').replace('```', '')
print(extract_class_code(aaa, "PathPlanning"))

class PathPlanning(SampleBasedAlgorithm):
    import time
    """
    {
    This PathPlanning algorithm integrates elements from informed sampling, heuristic-guided tree growth,
    and adaptive step sizes to improve efficiency, path quality, and robustness for grid-based path planning.
    It uses a Forest structure (a tree rooted at start vertex) with the following key features:
    - Goal-biased sampling: with a probability, samples the goal itself to push the exploration towards it.
    - Heuristic cost incorporates actual path cost + Euclidean distance to goal guiding vertex selection and rewiring.
    - Adaptive max_dist: reduces the step size as the tree grows closer to the goal, enabling finer exploration near goal.
    - Rewiring step inspired by RRT*: attempts to reduce costs by reconnecting vertices within a dynamically computed radius.
    - Early stopping as soon as the goal is connected within a tolerance radius.
    - Shortcut smoothing applied after extraction to produc

In [4]:
a,b = get_alg_new(code)

In [5]:
print(b)

This PathPlanning algorithm integrates elements from informed sampling, heuristic-guided tree growth,
    and adaptive step sizes to improve efficiency, path quality, and robustness for grid-based path planning.
    It uses a Forest structure (a tree rooted at start vertex) with the following key features:
    - Goal-biased sampling: with a probability, samples the goal itself to push the exploration towards it.
    - Heuristic cost incorporates actual path cost + Euclidean distance to goal guiding vertex selection and rewiring.
    - Adaptive max_dist: reduces the step size as the tree grows closer to the goal, enabling finer exploration near goal.
    - Rewiring step inspired by RRT*: attempts to reduce costs by reconnecting vertices within a dynamically computed radius.
    - Early stopping as soon as the goal is connected within a tolerance radius.
    - Shortcut smoothing applied after extraction to produce a shorter, smoother path.
    - Enforces a max computation time limit of 3

In [6]:
print(a.replace(b, ''))

class PathPlanning(SampleBasedAlgorithm):
    


In [7]:
print(a)

class PathPlanning(SampleBasedAlgorithm):
    


In [18]:
import sys

In [ ]:
def fb():
    c_name = sys._getframe().f_code.co_name
    print(c_name)

In [27]:
import ast

def get_called_helper_functions(class_code: str) -> set:
    # 파싱
    tree = ast.parse(class_code)
    
    class_def = None
    for node in tree.body:
        if isinstance(node, ast.ClassDef):
            class_def = node
            break

    if class_def is None:
        return set()

    # 1. 클래스 내부에 정의된 함수 이름 수집
    defined_methods = {f.name for f in class_def.body if isinstance(f, ast.FunctionDef)}

    # 2. 클래스 내부에서 호출된 메서드 이름 수집
    called_methods = set()

    class MethodCallVisitor(ast.NodeVisitor):
        def visit_Call(self, node):
            if isinstance(node.func, ast.Attribute) and isinstance(node.func.value, ast.Name):
                if node.func.value.id == "self":
                    called_methods.add(node.func.attr)
            self.generic_visit(node)

    MethodCallVisitor().visit(class_def)

    # 3. 정의되지 않은 메서드만 추려냄 = helper function
    helper_functions = called_methods - defined_methods
    return helper_functions, defined_methods

In [43]:
classs = '''
class RRT_Connect(SampleBasedAlgorithm):
    _graph: Forest
    _max_dist: float
    _iterations: int

    def __init__(self, services: Services, testing: BasicTesting = None) -> None:
        super().__init__(services, testing)
        
        self._graph = gen_forest(self._services, Vertex(self._get_grid().agent.position), Vertex(self._get_grid().goal.position), [])
        self._graph.edges_removable = False
        self._init_displays()
        
        self._max_dist = 10
        self._iterations = 10000

    # Helper Functions #
    # -----------------#

    

    # Overridden Implementation #
    # --------------------------#

    #@profile
    def _find_path_internal(self) -> None:

        for i in range(self._iterations):
            global n 
            n+=1

            if n == 100:
                break

            q_rand: Point = self._get_random_sample()

            if not self._extend(self._graph.root_vertices[0], q_rand) == 'trapped':
                self._extension_target = self._q_new
                if self._connect(self._graph.root_vertices[-1], self._q_new) == 'reached':
                    self._extract_path()
                    break
            self._graph.reverse_root_vertices()

            # visualization code
            self.key_frame()


'''

In [ ]:
classs='''
from typing import List

import torch
import numpy as np

from memory_profiler import profile

from algorithms.classic.sample_based.core.sample_based_algorithm import SampleBasedAlgorithm
from algorithms.basic_testing import BasicTesting
from simulator.services.services import Services
from structures import Point

from algorithms.classic.sample_based.core.vertex import Vertex
from algorithms.classic.sample_based.core.graph import gen_forest, Forest

n=0

class RRT_Connect(SampleBasedAlgorithm):
    _graph: Forest
    _max_dist: float
    _iterations: int

    def __init__(self, services: Services, testing: BasicTesting = None) -> None:
        super().__init__(services, testing)
        
        self._graph = gen_forest(self._services, Vertex(self._get_grid().agent.position), Vertex(self._get_grid().goal.position), [])
        self._graph.edges_removable = False
        self._init_displays()
        
        self._max_dist = 10
        self._iterations = 10000

    # Helper Functions #
    # -----------------#


    def _extract_path(self):

        # trace back
        path_mid_to_b: List[Vertex] = [self._q_new]
        defdd = self._extend(self._graph.root_vertices[0], q_rand)
        while len(path_mid_to_b[-1].parents) != 0:
            for parent in path_mid_to_b[-1].parents:
                path_mid_to_b.append(parent)
                break

        path_a_to_mid: List[Vertex] = [self._extension_target]

        while len(path_a_to_mid[-1].parents) != 0:
            for parent in path_a_to_mid[-1].parents:
                path_a_to_mid.append(parent)
                break

        path_a_to_mid.reverse()
        path = path_a_to_mid + path_mid_to_b

        if self._graph.root_vertices[0] is self._graph.root_vertex_goal:
            path.reverse()

        for p in path:
            self.move_agent(p.position)
            self.key_frame(ignore_key_frame_skip=True)


    def _get_nearest_vertex(self, graph_root_vertex: Vertex, q_sample: Point) -> Vertex:
        return self._graph.get_nearest_vertex([graph_root_vertex], q_sample)

    def _get_new_vertex(self, q_near: Vertex, q_sample: Point, max_dist) -> Vertex:
        dir = q_sample.to_tensor() - q_near.position.to_tensor()
        if torch.norm(dir) <= max_dist:
            return Vertex(q_sample)

        dir_normalized = dir / torch.norm(dir)
        q_new = Point.from_tensor(q_near.position.to_tensor() + max_dist * dir_normalized)
        return Vertex(q_new)

    # Overridden Implementation #
    # --------------------------#

    #@profile
    def _find_path_internal(self) -> None:

        for i in range(self._iterations):
            global n 
            n+=1

            if n == 100:
                break

            q_rand: Point = self._get_random_sample()
            abc = self._extend(self._graph.root_vertices[0], q_rand)
            if not self._extend(self._graph.root_vertices[0], q_rand) == 'trapped':
                self._extension_target = self._q_new
                if self._connect(self._graph.root_vertices[-1], self._q_new) == 'reached':
                    self._extract_path()
                    break
            self._graph.reverse_root_vertices()

            # visualization code
            self.key_frame()
'''

In [19]:
a,b = get_called_helper_functions(classs)
print(a)
print(b)

{'move_agent', '_init_displays', '_get_grid', 'key_frame'}
{'_extract_path', '_get_random_sample', '_extend', '__init__', '_get_nearest_vertex', '_connect', '_get_new_vertex', '_find_path_internal'}


In [29]:
fs

{}

In [39]:
import ast
import astunparse
import builtins

class FunctionParser(ast.NodeTransformer):
    def __init__(self, fs: dict, f_assigns: dict):
        super().__init__()
        self._fs = fs
        self._f_assigns = f_assigns
        self._defined_funcs = set()

        # 내장 함수 이름 목록 (예: range, len, super 등)
        self._builtin_names = set(dir(builtins))

        # 필터할 외부 util 함수 이름 (필요시 수동 추가 가능)
        self._common_known_funcs = {"Vertex", "gen_forest", "_get_grid", "_init_displays", "key_frame", "Point", "move_agent", }

    def visit_ClassDef(self, node):
        # 클래스 내부 함수 정의 수집
        for item in node.body:
            if isinstance(item, ast.FunctionDef):
                self._defined_funcs.add(item.name)
        self.generic_visit(node)
        return node

    def visit_Call(self, node):
        self.generic_visit(node)

        f_name = None
        if isinstance(node.func, ast.Name):
            f_name = node.func.id
        elif isinstance(node.func, ast.Attribute) and isinstance(node.func.value, ast.Name) and node.func.value.id == "self":
            f_name = node.func.attr

        if not f_name:
            return node

        # 필터링: 정의된 함수, 내장 함수, 공통 유틸 함수 제외
        if f_name in self._defined_funcs:
            return node
        if f_name in self._builtin_names:
            return node
        if f_name in self._common_known_funcs:
            return node

        f_sig = astunparse.unparse(node).strip()
        self._fs[f_name] = f_sig
        return node

    def visit_Assign(self, node):
        self.generic_visit(node)

        if isinstance(node.value, ast.Call):
            func = node.value.func
            if isinstance(func, ast.Attribute) and isinstance(func.value, ast.Name) and func.value.id == "self":
                f_name = func.attr
                if f_name in self._defined_funcs or f_name in self._builtin_names or f_name in self._common_known_funcs:
                    return node
                assign_str = astunparse.unparse(node).strip()
                self._f_assigns[f_name] = assign_str
        return node


In [44]:
tree = ast.parse(classs)
fs = {}
f_assigns = {}

FunctionParser(fs, f_assigns).visit(tree)

In [45]:
fs

{'_get_random_sample': 'self._get_random_sample()',
 '_extend': 'self._extend(self._graph.root_vertices[0], q_rand)',
 '_connect': 'self._connect(self._graph.root_vertices[(- 1)], self._q_new)',
 '_extract_path': 'self._extract_path()'}

In [46]:
f_assigns

{}